# : פלט הקוד

#  .א) מילון המכיל מרחק וזמן מת"א לכל יעד שניתן בשאלה וכן, קווי אורך ורוחב של כל יעד

#  .ב) הדפסה של שלושת הערים הכי רחוקות מת"א

# .במקום המתאיםAPI-הערה- יש לשים את ה

#   הערה 2- זמני הנסיעה נלקחו מערך השניות במשך הנסיעה והומרו לשעות ולדקות

In [42]:
# סיפריות

import json 
import requests
import pprint

try:
    # וקליטת קובץ הערים לרשימה Distance_Matrix-שימוש ב

    api_key = "Put your API here"# השמת המפתח של המשתמש
    destinations=open("dests.txt",encoding='utf-8')
    destinations_List=list()

    try:
        for i in destinations:
            i=i.strip()
            destinations_List.append(i)
        url='https://maps.googleapis.com/maps/api/distancematrix/json?origins=תל אביב&destinations=%s|%s|%s|%s|%s&key=%s'% (destinations_List[0],destinations_List[1],destinations_List[2],destinations_List[3],destinations_List[4],api_key)

        if(requests.get(url).status_code==200):
            response = requests.get(url).json()  
            # יצירת מילון של הערים 
            Cities_List=response['destination_addresses']
            Cities_Dict=dict.fromkeys(Cities_List)

            # מציאת מרחק וזמן יעד מתל אביב לכל עיר
            dis_dur=response['rows']

            Index_City=0
            dur_dis_list=list() # יצירת רשימה של המרחק ושל זמן היעד מתל אביב לכל עיר

            for city in Cities_List:
                City_Dis=dis_dur[0]['elements'][Index_City]['distance']['text']
                City_Dur=dis_dur[0]['elements'][Index_City]['duration']['value']
                if((City_Dur/3600)<1):
                    City_Dur=int(City_Dur/60)
                    City_Dur=str(City_Dur) + ' mins'
                else:
                    hours=int(City_Dur/3600)
                    mins=int(((City_Dur%3600.)/3600)*60)
                    City_Dur=str(hours)+' hours '+str(mins)+' mins'
                First_Tuple=City_Dis,City_Dur
                dur_dis_list.append(First_Tuple)
                Index_City=Index_City+1

            # השמת זמן הגעה ומרחק מת"א לכל עיר במילון
            Index_dur_dis=0
            for city in Cities_List:
                Cities_Dict[city]=dur_dis_list[Index_dur_dis]
                Index_dur_dis=Index_dur_dis+1


            # Geocoding-שימוש ב 
            # מציאת קווי אורך ורוחב לכל עיר

            lat_lon_list=list()
            for city in destinations_List:  
                url_2='https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s'% (city,api_key)
                if(requests.get(url_2).status_code==200):
                    response_2 = requests.get(url_2).json()  
                    lat_lon=response_2['results']
                    lat_location='lat :'+str(lat_lon[0]['geometry']['location']['lat'])
                    lng_location='lng :'+str(lat_lon[0]['geometry']['location']['lng'])
                    lat_lon_location=lat_location,lng_location
                    lat_lon_list.append(lat_lon_location)
                else:
                    print('HTTP error :',requests.get(url_2).status_code )
                    break
                    
            # הוספת קווי אורך ורוחב למילון 
            Index_lat_lon=0
            for city in Cities_List:
                Cities_Dict[city]=Cities_Dict[city]+lat_lon_list[Index_lat_lon]
                Index_lat_lon=Index_lat_lon+1

            # הדפסת המילון הסופי
            pprint.pprint(Cities_Dict) 


            # מציאת 3 הערים הכי רחוקות מת"א

            # יצירת צמדים - ערים ומרחק בק"מ מתל אביב בכדי למיין ולהציג את שלושת הערים הכי רחוקות מתל אביב
            Km_City_List=list()

            for city in Cities_List:
                City_Km_Tuple=city,Cities_Dict[city][0]
                Km_City_List.append(City_Km_Tuple)


            #  פונקציה שמחזירה את האיבר השני ומחזירה את מספר הק"מ
            def Km_Dis(city):
                Km_Only=city[1]
                Km_Only=Km_Only.split(' ')
                Km_Only=str(Km_Only[0])
                if(Km_Only.find(',')==-1):
                    return float(Km_Only)
                else:
                    Km_Only=Km_Only.replace(',','')
                    return float(Km_Only)

            Km_City_List.sort(key=Km_Dis, reverse=True)

            # הדפסת שלושת הערים הכי רחוקות מתל-אביב כולל המרחק שלהם בק"מ

            print("\n","\u0332".join("The 3 furthest cities from Tel Aviv are :"), "\n")
            pprint.pprint(Km_City_List[0:3])
        else:
            print('HTTP error :',requests.get(url).status_code)
    except:
        if(requests.get(url_2).status_code!=200):
            pass
        else:
            print('Somthing wrong with the request or with the cities (city does not exist or wrong city spelling or cities number is not 5)')
except:
    print('Somthing wrong with your text file')

{'Amsterdam, Netherlands': ('4,528 km',
                            '47 hours 25 mins',
                            'lat :52.3675734',
                            'lng :4.9041389'),
 'Basel, Switzerland': ('4,093 km',
                        '43 hours 34 mins',
                        'lat :47.5595986',
                        'lng :7.5885761'),
 'Kotor, Montenegro': ('3,015 km',
                       '35 hours 35 mins',
                       'lat :42.424662',
                       'lng :18.771234'),
 'Valletta, Malta': ('3,795 km',
                     '50 hours 40 mins',
                     'lat :35.8989085',
                     'lng :14.5145528'),
 'İstanbul, Turkey': ('1,815 km',
                      '20 hours 32 mins',
                      'lat :41.0082376',
                      'lng :28.9783589')}

 T̲h̲e̲ ̲3̲ ̲f̲u̲r̲t̲h̲e̲s̲t̲ ̲c̲i̲t̲i̲e̲s̲ ̲f̲r̲o̲m̲ ̲T̲e̲l̲ ̲A̲v̲i̲v̲ ̲a̲r̲e̲ ̲: 

[('Amsterdam, Netherlands', '4,528 km'),
 ('Basel, Switzerland', '4,093 km'),
 ('Valletta, 